# Environment

## Imports

In [176]:
:dep polars = { version = "0.46", features = [ "lazy", "cum_agg", "list_arithmetic" ] }
:dep nalgebra
:dep argmin
:dep argmin-math = { version = "0.4", features = ["nalgebra_latest", "vec"] }
:dep itertools
:dep plotters = { version = "0.3", features = ["evcxr", "all_series"] }

[Polars](https://pola.rs/) is a new DataFrame library meant to replace pandas, which is written in Rust. It has both a Python and a Rust interface

It's by far the largest library here, so it will take some time to compile. I have only enabled the features that I will use.

In [177]:
use polars::prelude::*;

# Problem 1

## a)

Initialize data.

In [178]:
let df: DataFrame = df!(
    "rate_type" => ["cash", "cash", "forwards", "forwards", "forwards", "swaps", "swaps", "swaps", "swaps", "swaps"],
    "maturity" => [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0],
    "inputs" => [0.03180, 0.03222, 0.03261, 0.03290, 0.03345, 0.03405, 0.03442, 0.03350, 0.03300, 0.03541],
    "bond_cash_flow" => [3, 3, 3, 3, 3, 3, 3, 3, 103, 0],
)?;

df

shape: (10, 4)
┌───────────┬──────────┬─────────┬────────────────┐
│ rate_type ┆ maturity ┆ inputs  ┆ bond_cash_flow │
│ ---       ┆ ---      ┆ ---     ┆ ---            │
│ str       ┆ f64      ┆ f64     ┆ i32            │
╞═══════════╪══════════╪═════════╪════════════════╡
│ cash      ┆ 1.0      ┆ 0.0318  ┆ 3              │
│ cash      ┆ 2.0      ┆ 0.03222 ┆ 3              │
│ forwards  ┆ 3.0      ┆ 0.03261 ┆ 3              │
│ forwards  ┆ 4.0      ┆ 0.0329  ┆ 3              │
│ forwards  ┆ 5.0      ┆ 0.03345 ┆ 3              │
│ swaps     ┆ 6.0      ┆ 0.03405 ┆ 3              │
│ swaps     ┆ 7.0      ┆ 0.03442 ┆ 3              │
│ swaps     ┆ 8.0      ┆ 0.0335  ┆ 3              │
│ swaps     ┆ 9.0      ┆ 0.033   ┆ 103            │
│ swaps     ┆ 10.0     ┆ 0.03541 ┆ 0              │
└───────────┴──────────┴─────────┴────────────────┘

Place the inputs into their appropriate curves.

In [179]:
let df = df
    .lazy()
    .with_column(
        when(col("rate_type").eq(lit("cash")))
            .then(col("inputs"))
            .otherwise(lit(NULL))
            .alias("zero_curve"),
    )
    .with_column(
        when(col("rate_type").eq(lit("forwards")))
            .then(col("inputs"))
            .otherwise(lit(NULL))
            .alias("forward_curve"),
    )
    .with_column(
        when(col("rate_type").eq(lit("swaps")))
            .then(col("inputs"))
            .otherwise(lit(NULL))
            .alias("par_curve"),
    )
    .collect()?;

df

shape: (10, 7)
┌───────────┬──────────┬─────────┬────────────────┬────────────┬───────────────┬───────────┐
│ rate_type ┆ maturity ┆ inputs  ┆ bond_cash_flow ┆ zero_curve ┆ forward_curve ┆ par_curve │
│ ---       ┆ ---      ┆ ---     ┆ ---            ┆ ---        ┆ ---           ┆ ---       │
│ str       ┆ f64      ┆ f64     ┆ i32            ┆ f64        ┆ f64           ┆ f64       │
╞═══════════╪══════════╪═════════╪════════════════╪════════════╪═══════════════╪═══════════╡
│ cash      ┆ 1.0      ┆ 0.0318  ┆ 3              ┆ 0.0318     ┆ null          ┆ null      │
│ cash      ┆ 2.0      ┆ 0.03222 ┆ 3              ┆ 0.03222    ┆ null          ┆ null      │
│ forwards  ┆ 3.0      ┆ 0.03261 ┆ 3              ┆ null       ┆ 0.03261       ┆ null      │
│ forwards  ┆ 4.0      ┆ 0.0329  ┆ 3              ┆ null       ┆ 0.0329        ┆ null      │
│ forwards  ┆ 5.0      ┆ 0.03345 ┆ 3              ┆ null       ┆ 0.03345       ┆ null      │
│ swaps     ┆ 6.0      ┆ 0.03405 ┆ 3              ┆ nul

Bootstrap forwards from zero curve.

In [180]:
fn bootstrap_forward(mut df: DataFrame) -> PolarsResult<DataFrame> {
    let mut bootstrapped_forward_curve = match df.column("forward_curve") {
        Ok(col) => col.f64()?.to_vec(),

        // forward curve does not exist, initialize with first zero curve value
        Err(_) => {
            let mut b = vec![None; df.height()];

            b[0] = Some(df.column("zero_curve")?.f64()?.get(0).unwrap());

            b
        }
    };

    // initialize with value with zero curve, since zero = forward for maturity = 1

    bootstrapped_forward_curve[0] = Some(df.column("zero_curve")?.f64()?.get(0).unwrap());

    // bootstrap from start of null

    let start_idx = bootstrapped_forward_curve
        .iter()
        .position(|&x| x.is_none())
        .unwrap();

    for i in start_idx..df.height() {
        let zero_val_opt = df.column("zero_curve")?.f64()?.get(i);

        bootstrapped_forward_curve[i] = if let Some(zero_val) = zero_val_opt {
            let maturity_val = df.column("maturity")?.f64()?.get(i).unwrap();

            let previous_forward_vals = &bootstrapped_forward_curve[..i];

            let prod: f64 = previous_forward_vals
                .iter()
                .map(|x| 1.0 / (1.0 + x.unwrap_or(0.0)))
                .product();

            Some((zero_val + 1.0).powf(maturity_val) * prod - 1.0)
        } else {
            None
        };
    }

    let bootstrapped_series = Column::new(
        "bootstrapped_forward_curve".into(),
        bootstrapped_forward_curve,
    );

    let forward_series = df.column("forward_curve")?;

    let mask = forward_series.is_not_null();

    let updated_forward_series = forward_series.zip_with(&mask, &bootstrapped_series)?;

    df.replace(
        "forward_curve",
        updated_forward_series.as_series().unwrap().clone(),
    )?;

    Ok(df)
}

In [181]:
let df = bootstrap_forward(df)?;
df

shape: (10, 7)
┌───────────┬──────────┬─────────┬────────────────┬────────────┬───────────────┬───────────┐
│ rate_type ┆ maturity ┆ inputs  ┆ bond_cash_flow ┆ zero_curve ┆ forward_curve ┆ par_curve │
│ ---       ┆ ---      ┆ ---     ┆ ---            ┆ ---        ┆ ---           ┆ ---       │
│ str       ┆ f64      ┆ f64     ┆ i32            ┆ f64        ┆ f64           ┆ f64       │
╞═══════════╪══════════╪═════════╪════════════════╪════════════╪═══════════════╪═══════════╡
│ cash      ┆ 1.0      ┆ 0.0318  ┆ 3              ┆ 0.0318     ┆ 0.0318        ┆ null      │
│ cash      ┆ 2.0      ┆ 0.03222 ┆ 3              ┆ 0.03222    ┆ 0.03264       ┆ null      │
│ forwards  ┆ 3.0      ┆ 0.03261 ┆ 3              ┆ null       ┆ 0.03261       ┆ null      │
│ forwards  ┆ 4.0      ┆ 0.0329  ┆ 3              ┆ null       ┆ 0.0329        ┆ null      │
│ forwards  ┆ 5.0      ┆ 0.03345 ┆ 3              ┆ null       ┆ 0.03345       ┆ null      │
│ swaps     ┆ 6.0      ┆ 0.03405 ┆ 3              ┆ nul

Create the discount curve from the forward curve.

In [182]:
let df = df
    .lazy()
    .with_column(
        when(col("forward_curve").is_not_null())
            .then(lit(1.0) / (lit(1.0) + col("forward_curve")).cum_prod(false))
            .otherwise(lit(NULL))
            .alias("discount_curve"),
    )
    .collect()?;

df

shape: (10, 8)
┌───────────┬──────────┬─────────┬─────────────┬────────────┬─────────────┬───────────┬────────────┐
│ rate_type ┆ maturity ┆ inputs  ┆ bond_cash_f ┆ zero_curve ┆ forward_cur ┆ par_curve ┆ discount_c │
│ ---       ┆ ---      ┆ ---     ┆ low         ┆ ---        ┆ ve          ┆ ---       ┆ urve       │
│ str       ┆ f64      ┆ f64     ┆ ---         ┆ f64        ┆ ---         ┆ f64       ┆ ---        │
│           ┆          ┆         ┆ i32         ┆            ┆ f64         ┆           ┆ f64        │
╞═══════════╪══════════╪═════════╪═════════════╪════════════╪═════════════╪═══════════╪════════════╡
│ cash      ┆ 1.0      ┆ 0.0318  ┆ 3           ┆ 0.0318     ┆ 0.0318      ┆ null      ┆ 0.96918    │
│ cash      ┆ 2.0      ┆ 0.03222 ┆ 3           ┆ 0.03222    ┆ 0.03264     ┆ null      ┆ 0.938546   │
│ forwards  ┆ 3.0      ┆ 0.03261 ┆ 3           ┆ null       ┆ 0.03261     ┆ null      ┆ 0.908906   │
│ forwards  ┆ 4.0      ┆ 0.0329  ┆ 3           ┆ null       ┆ 0.0329      ┆ 

Fill in the par curve from the discount curve.

In [183]:
let df = df
    .lazy()
    .with_column(
        when(col("discount_curve").is_not_null())
            .then((lit(1.0) - col("discount_curve")) / (col("discount_curve").cum_sum(false)))
            .otherwise(col("par_curve"))
            .alias("par_curve"),
    )
    .collect()?;

df

shape: (10, 8)
┌───────────┬──────────┬─────────┬─────────────┬────────────┬─────────────┬───────────┬────────────┐
│ rate_type ┆ maturity ┆ inputs  ┆ bond_cash_f ┆ zero_curve ┆ forward_cur ┆ par_curve ┆ discount_c │
│ ---       ┆ ---      ┆ ---     ┆ low         ┆ ---        ┆ ve          ┆ ---       ┆ urve       │
│ str       ┆ f64      ┆ f64     ┆ ---         ┆ f64        ┆ ---         ┆ f64       ┆ ---        │
│           ┆          ┆         ┆ i32         ┆            ┆ f64         ┆           ┆ f64        │
╞═══════════╪══════════╪═════════╪═════════════╪════════════╪═════════════╪═══════════╪════════════╡
│ cash      ┆ 1.0      ┆ 0.0318  ┆ 3           ┆ 0.0318     ┆ 0.0318      ┆ 0.0318    ┆ 0.96918    │
│ cash      ┆ 2.0      ┆ 0.03222 ┆ 3           ┆ 0.03222    ┆ 0.03264     ┆ 0.032213  ┆ 0.938546   │
│ forwards  ┆ 3.0      ┆ 0.03261 ┆ 3           ┆ null       ┆ 0.03261     ┆ 0.032341  ┆ 0.908906   │
│ forwards  ┆ 4.0      ┆ 0.0329  ┆ 3           ┆ null       ┆ 0.0329      ┆ 

Bootstrap the rest of the discount curve from the par curve.

In [184]:
fn bootstrap_discount(mut df: DataFrame) -> PolarsResult<DataFrame> {
    let mut bootstrapped_discount_curve = df.column("discount_curve")?.f64()?.to_vec();

    let start_idx = bootstrapped_discount_curve
        .iter()
        .position(|&x| x.is_none())
        .unwrap();

    for i in start_idx..bootstrapped_discount_curve.len() {
        let par_val_opt = df.column("par_curve")?.f64()?.get(i);

        bootstrapped_discount_curve[i] = if let Some(par_val) = par_val_opt {
            let previous_discount_vals = &bootstrapped_discount_curve[..i];

            let sum: f64 = previous_discount_vals
                .iter()
                .map(|x| x.unwrap_or(0.0))
                .sum();

            Some((1.0 - par_val * sum) / (1.0 + par_val))
        } else {
            None
        };
    }

    df.replace(
        "discount_curve",
        Series::new("discount_curve".into(), bootstrapped_discount_curve),
    )?;

    Ok(df)
}

In [185]:
let df = bootstrap_discount(df)?;
df

shape: (10, 8)
┌───────────┬──────────┬─────────┬─────────────┬────────────┬─────────────┬───────────┬────────────┐
│ rate_type ┆ maturity ┆ inputs  ┆ bond_cash_f ┆ zero_curve ┆ forward_cur ┆ par_curve ┆ discount_c │
│ ---       ┆ ---      ┆ ---     ┆ low         ┆ ---        ┆ ve          ┆ ---       ┆ urve       │
│ str       ┆ f64      ┆ f64     ┆ ---         ┆ f64        ┆ ---         ┆ f64       ┆ ---        │
│           ┆          ┆         ┆ i32         ┆            ┆ f64         ┆           ┆ f64        │
╞═══════════╪══════════╪═════════╪═════════════╪════════════╪═════════════╪═══════════╪════════════╡
│ cash      ┆ 1.0      ┆ 0.0318  ┆ 3           ┆ 0.0318     ┆ 0.0318      ┆ 0.0318    ┆ 0.96918    │
│ cash      ┆ 2.0      ┆ 0.03222 ┆ 3           ┆ 0.03222    ┆ 0.03264     ┆ 0.032213  ┆ 0.938546   │
│ forwards  ┆ 3.0      ┆ 0.03261 ┆ 3           ┆ null       ┆ 0.03261     ┆ 0.032341  ┆ 0.908906   │
│ forwards  ┆ 4.0      ┆ 0.0329  ┆ 3           ┆ null       ┆ 0.0329      ┆ 

Get the zero curve from the discount curve.

In [186]:
let df = df
    .lazy()
    .with_column(
        when(col("zero_curve").is_null())
            .then(col("discount_curve").pow(lit(-1.0) / col("maturity")) - lit(1.0))
            .otherwise(col("zero_curve"))
            .alias("zero_curve"),
    )
    .collect()?;

df

shape: (10, 8)
┌───────────┬──────────┬─────────┬─────────────┬────────────┬─────────────┬───────────┬────────────┐
│ rate_type ┆ maturity ┆ inputs  ┆ bond_cash_f ┆ zero_curve ┆ forward_cur ┆ par_curve ┆ discount_c │
│ ---       ┆ ---      ┆ ---     ┆ low         ┆ ---        ┆ ve          ┆ ---       ┆ urve       │
│ str       ┆ f64      ┆ f64     ┆ ---         ┆ f64        ┆ ---         ┆ f64       ┆ ---        │
│           ┆          ┆         ┆ i32         ┆            ┆ f64         ┆           ┆ f64        │
╞═══════════╪══════════╪═════════╪═════════════╪════════════╪═════════════╪═══════════╪════════════╡
│ cash      ┆ 1.0      ┆ 0.0318  ┆ 3           ┆ 0.0318     ┆ 0.0318      ┆ 0.0318    ┆ 0.96918    │
│ cash      ┆ 2.0      ┆ 0.03222 ┆ 3           ┆ 0.03222    ┆ 0.03264     ┆ 0.032213  ┆ 0.938546   │
│ forwards  ┆ 3.0      ┆ 0.03261 ┆ 3           ┆ 0.03235    ┆ 0.03261     ┆ 0.032341  ┆ 0.908906   │
│ forwards  ┆ 4.0      ┆ 0.0329  ┆ 3           ┆ 0.032487   ┆ 0.0329      ┆ 

Continue bootstrapping forward curve with finished zero curve.

In [187]:
let df = bootstrap_forward(df)?;
df

shape: (10, 8)
┌───────────┬──────────┬─────────┬─────────────┬────────────┬─────────────┬───────────┬────────────┐
│ rate_type ┆ maturity ┆ inputs  ┆ bond_cash_f ┆ zero_curve ┆ forward_cur ┆ par_curve ┆ discount_c │
│ ---       ┆ ---      ┆ ---     ┆ low         ┆ ---        ┆ ve          ┆ ---       ┆ urve       │
│ str       ┆ f64      ┆ f64     ┆ ---         ┆ f64        ┆ ---         ┆ f64       ┆ ---        │
│           ┆          ┆         ┆ i32         ┆            ┆ f64         ┆           ┆ f64        │
╞═══════════╪══════════╪═════════╪═════════════╪════════════╪═════════════╪═══════════╪════════════╡
│ cash      ┆ 1.0      ┆ 0.0318  ┆ 3           ┆ 0.0318     ┆ 0.0318      ┆ 0.0318    ┆ 0.96918    │
│ cash      ┆ 2.0      ┆ 0.03222 ┆ 3           ┆ 0.03222    ┆ 0.03264     ┆ 0.032213  ┆ 0.938546   │
│ forwards  ┆ 3.0      ┆ 0.03261 ┆ 3           ┆ 0.03235    ┆ 0.03261     ┆ 0.032341  ┆ 0.908906   │
│ forwards  ┆ 4.0      ┆ 0.0329  ┆ 3           ┆ 0.032487   ┆ 0.0329      ┆ 

## b)

Get present value of the bond cash flows.

In [188]:
let pv = (col("discount_curve") * col("bond_cash_flow"))
    .sum()
    .alias("pv");

df.clone().lazy().select([pv.clone()]).collect()?

shape: (1, 1)
┌───────────┐
│ pv        │
│ ---       │
│ f64       │
╞═══════════╡
│ 97.699413 │
└───────────┘

## c)

We will need to calculate DV01, the dollar value of one basis point:

![dv01](https://latex.codecogs.com/svg.image?&space;DV01=-\frac{\Delta&space;P}{10,000\cdot\Delta&space;y})

In [189]:
const BP: f64 = 0.0001;

let discount_curve_up = (col("discount_curve") / lit(1.0 + BP)).alias("discount_curve_up");

let discount_curve_down = (col("discount_curve") * lit(1.0 + BP)).alias("discount_curve_down");

let pv_up = ((discount_curve_up.clone() * col("bond_cash_flow")).sum()).alias("pv_up");

let pv_down = ((discount_curve_down.clone() * col("bond_cash_flow")).sum()).alias("pv_down");

let dv01 = ((pv_up.clone() - pv_down.clone()) / lit(2.0)).alias("dv01");

df.clone().lazy().select([dv01.clone()]).collect()?

shape: (1, 1)
┌───────────┐
│ dv01      │
│ ---       │
│ f64       │
╞═══════════╡
│ -0.009769 │
└───────────┘

We also need to calculate duration, a measure of linear risk.

In [190]:
let duration =
    ((pv_up.clone() - pv_down.clone()) / (lit(2.0 * BP) * pv.clone())).alias("duration");

df.clone().lazy().select([duration.clone()]).collect()?

shape: (1, 1)
┌──────────┐
│ duration │
│ ---      │
│ f64      │
╞══════════╡
│ -0.99995 │
└──────────┘

One more thing: we need to generate our new discount curve from the forward curve.

In [191]:
let discount_curve = (lit(1.0) / (lit(1.0) + col("forward_curve")).cum_prod(false).implode())
    .alias("discount_curve");

df.clone()
    .lazy()
    .select([discount_curve.clone()])
    .collect()?

shape: (1, 1)
┌─────────────────────────────────┐
│ discount_curve                  │
│ ---                             │
│ list[f64]                       │
╞═════════════════════════════════╡
│ [0.96918, 0.938546, … 0.703542… │
└─────────────────────────────────┘

Since we are creating ten different scenarios altering each row, we'll need the row index.

In [192]:
let df = if df.column("row").is_err() {
    df.with_row_index("row".into(), None)?
} else {
    df
};

df

shape: (10, 9)
┌─────┬───────────┬──────────┬─────────┬───┬────────────┬───────────────┬───────────┬──────────────┐
│ row ┆ rate_type ┆ maturity ┆ inputs  ┆ … ┆ zero_curve ┆ forward_curve ┆ par_curve ┆ discount_cur │
│ --- ┆ ---       ┆ ---      ┆ ---     ┆   ┆ ---        ┆ ---           ┆ ---       ┆ ve           │
│ u32 ┆ str       ┆ f64      ┆ f64     ┆   ┆ f64        ┆ f64           ┆ f64       ┆ ---          │
│     ┆           ┆          ┆         ┆   ┆            ┆               ┆           ┆ f64          │
╞═════╪═══════════╪══════════╪═════════╪═══╪════════════╪═══════════════╪═══════════╪══════════════╡
│ 0   ┆ cash      ┆ 1.0      ┆ 0.0318  ┆ … ┆ 0.0318     ┆ 0.0318        ┆ 0.0318    ┆ 0.96918      │
│ 1   ┆ cash      ┆ 2.0      ┆ 0.03222 ┆ … ┆ 0.03222    ┆ 0.03264       ┆ 0.032213  ┆ 0.938546     │
│ 2   ┆ forwards  ┆ 3.0      ┆ 0.03261 ┆ … ┆ 0.03235    ┆ 0.03261       ┆ 0.032341  ┆ 0.908906     │
│ 3   ┆ forwards  ┆ 4.0      ┆ 0.0329  ┆ … ┆ 0.032487   ┆ 0.0329        ┆ 0.

We will now generate ten different scenarios for incrementing each part of the forward curve.

In [193]:
let scenarios_df = (0..df.height())
    .map(|i| {
        df.clone()
            .lazy()
            .with_column(
                when(col("row").eq(lit(i as u32)))
                    .then(col("forward_curve") + lit(0.001))
                    .otherwise(col("forward_curve"))
                    .alias("forward_curve"),
            )
            .with_column(
                (lit(1.0) / (lit(1.0) + col("forward_curve")).cum_prod(false))
                    .alias("discount_curve"),
            )
            .collect()
            .unwrap()
    })
    .collect::<Vec<_>>();

scenarios_df[0]

shape: (10, 9)
┌─────┬───────────┬──────────┬─────────┬───┬────────────┬───────────────┬───────────┬──────────────┐
│ row ┆ rate_type ┆ maturity ┆ inputs  ┆ … ┆ zero_curve ┆ forward_curve ┆ par_curve ┆ discount_cur │
│ --- ┆ ---       ┆ ---      ┆ ---     ┆   ┆ ---        ┆ ---           ┆ ---       ┆ ve           │
│ u32 ┆ str       ┆ f64      ┆ f64     ┆   ┆ f64        ┆ f64           ┆ f64       ┆ ---          │
│     ┆           ┆          ┆         ┆   ┆            ┆               ┆           ┆ f64          │
╞═════╪═══════════╪══════════╪═════════╪═══╪════════════╪═══════════════╪═══════════╪══════════════╡
│ 0   ┆ cash      ┆ 1.0      ┆ 0.0318  ┆ … ┆ 0.0318     ┆ 0.0328        ┆ 0.0318    ┆ 0.968242     │
│ 1   ┆ cash      ┆ 2.0      ┆ 0.03222 ┆ … ┆ 0.03222    ┆ 0.03264       ┆ 0.032213  ┆ 0.937637     │
│ 2   ┆ forwards  ┆ 3.0      ┆ 0.03261 ┆ … ┆ 0.03235    ┆ 0.03261       ┆ 0.032341  ┆ 0.908026     │
│ 3   ┆ forwards  ┆ 4.0      ┆ 0.0329  ┆ … ┆ 0.032487   ┆ 0.0329        ┆ 0.

Now we will calculate our different values for our metrics.

In [194]:
let scenarios = concat(
    scenarios_df
        .iter()
        .enumerate()
        .map(|(i, scenario)| {
            scenario.clone().lazy().select([
                lit(i as u32).alias("increment"),
                discount_curve.clone(),
                pv.clone(),
                dv01.clone(),
                duration.clone(),
            ])
        })
        .collect::<Vec<_>>(),
    UnionArgs::default(),
)?
.collect()?;

scenarios

shape: (10, 5)
┌───────────┬─────────────────────────────────┬───────────┬───────────┬──────────┐
│ increment ┆ discount_curve                  ┆ pv        ┆ dv01      ┆ duration │
│ ---       ┆ ---                             ┆ ---       ┆ ---       ┆ ---      │
│ i32       ┆ list[f64]                       ┆ f64       ┆ f64       ┆ f64      │
╞═══════════╪═════════════════════════════════╪═══════════╪═══════════╪══════════╡
│ 0         ┆ [0.968242, 0.937637, … 0.70286… ┆ 97.604816 ┆ -0.00976  ┆ -0.99995 │
│ 1         ┆ [0.96918, 0.937638, … 0.702861… ┆ 97.607706 ┆ -0.00976  ┆ -0.99995 │
│ 2         ┆ [0.96918, 0.938546, … 0.702861… ┆ 97.610427 ┆ -0.009761 ┆ -0.99995 │
│ 3         ┆ [0.96918, 0.938546, … 0.702861… ┆ 97.61309  ┆ -0.009761 ┆ -0.99995 │
│ 4         ┆ [0.96918, 0.938546, … 0.702861… ┆ 97.615687 ┆ -0.009761 ┆ -0.99995 │
│ 5         ┆ [0.96918, 0.938546, … 0.702867… ┆ 97.618808 ┆ -0.009761 ┆ -0.99995 │
│ 6         ┆ [0.96918, 0.938546, … 0.702864… ┆ 97.620792 ┆ -0.009762 ┆ 

The forward change is 10 bps, which causes a couple bps change in the discount curve, a few cents change in the price, a hundredth of a bp change in DV01, and no change in duration.

The further in time the increment goes:
- the higher the final discount is pushed up, approaching the original discount
- the higher the price becomes, approaching the original price
- the higher the duration becomes, approaching the original duration

## d)

Now we'll move all of the forward rates by 10 bps.

In [195]:
let forward_curve_up = col("forward_curve") + lit(10.0 * BP);

let forward_curve_down = col("forward_curve") - lit(10.0 * BP);

// redefine discount curve shift from forward curve

let discount_curve_up = (lit(1.0) / (lit(1.0) + forward_curve_up.clone()).cum_prod(false));

let discount_curve_down = (lit(1.0) / (lit(1.0) + forward_curve_down.clone()).cum_prod(false));

let pv_up = ((discount_curve_up.clone() * col("bond_cash_flow")).sum()).alias("pv_up");

let pv_down = ((discount_curve_down.clone() * col("bond_cash_flow")).sum()).alias("pv_down");

df.clone()
    .lazy()
    .select([pv_up.clone(), pv_down.clone()])
    .collect()?

shape: (1, 2)
┌───────────┬───────────┐
│ pv_up     ┆ pv_down   │
│ ---       ┆ ---       │
│ f64       ┆ f64       │
╞═══════════╪═══════════╡
│ 96.945739 ┆ 98.460125 │
└───────────┴───────────┘

And here is the convexity of the bond.

In [196]:
let convexity = ((pv_up.clone() + pv_down.clone() - lit(2.0) * pv.clone())
    / (pv.clone() * (lit(10.0 * BP).pow(2))))
.alias("convexity");

df.clone().lazy().select([convexity.clone()]).collect()?

shape: (1, 1)
┌───────────┐
│ convexity │
│ ---       │
│ f64       │
╞═══════════╡
│ 72.040424 │
└───────────┘

## e)

We will use linear interpolation to compute the 30 month (2.5 year) forward price.

In [197]:
let discount_curve = df
    .clone()
    .column("discount_curve")?
    .f64()?
    .to_vec_null_aware()
    .left()
    .unwrap();

let discount30 =
    discount_curve[1] + (discount_curve[2] - discount_curve[1]) * (2.5 - 2.0) / (3.0 - 2.0);

df.clone()
    .lazy()
    .select([pv.clone()])
    .collect()?
    .column("pv")?
    .f64()?
    .get(0)
    .unwrap()
    / discount30

105.76665188731965

# Problem 2

In [ ]:
use nalgebra::{DMatrix, DVector}
use plotters::prelude::*;
use polars::prelude::*;